<a href="https://colab.research.google.com/github/rajy4683/EVAP2/blob/master/ENDS9_Attn_OpenBookQa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Mon Jan  4 13:19:13 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%%bash
python -m spacy download en
python -m spacy download de

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
  Created wheel for de-core-news-sm: filename=de_core_news_sm-2.2.5-cp36-none-any.whl size=14907057 sha256=1f7af03100f41afebb12a92ccfd7d24bc6b077c81264e13b3f5d0e33a68c9ec4
  Stored in directory: /tmp/pip-ephem-wheel-cache-isigl0rb/wheels/ba/3f/ed/d4aa8e45e7191b7f32db4bfad565e7da1edbf05c916ca7a1ca
Successfully built de-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/de_core_news_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/de
You can now load the model via spacy.load('de')


**Restart Notebook**

# 3 - Neural Machine Translation by Jointly Learning to Align and Translate

In this third notebook on sequence-to-sequence models using PyTorch and TorchText, we'll be implementing the model from [Neural Machine Translation by Jointly Learning to Align and Translate](https://arxiv.org/abs/1409.0473). This model achives our best perplexity yet, ~27 compared to ~34 for the previous model.

## Introduction

As a reminder, here is the general encoder-decoder model:

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq1.png?raw=1)

In the previous model, our architecture was set-up in a way to reduce "information compression" by explicitly passing the context vector, $z$, to the decoder at every time-step and by passing both the context vector and embedded input word, $d(y_t)$, along with the hidden state, $s_t$, to the linear layer, $f$, to make a prediction.

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq7.png?raw=1)

Even though we have reduced some of this compression, our context vector still needs to contain all of the information about the source sentence. The model implemented in this notebook avoids this compression by allowing the decoder to look at the entire source sentence (via its hidden states) at each decoding step! How does it do this? It uses *attention*. 

Attention works by first, calculating an attention vector, $a$, that is the length of the source sentence. The attention vector has the property that each element is between 0 and 1, and the entire vector sums to 1. We then calculate a weighted sum of our source sentence hidden states, $H$, to get a weighted source vector, $w$. 

$$w = \sum_{i}a_ih_i$$

We calculate a new weighted source vector every time-step when decoding, using it as input to our decoder RNN as well as the linear layer to make a prediction. We'll explain how to do all of this during the session.

## Preparing Data

Again, the preparation is similar to last time.

First we import all the required modules.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator

import spacy
import numpy as np
import pandas as pd
import random
import math
import time
import json

Set the random seeds for reproducability.

In [ ]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

Load the German and English spaCy models.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
spacy_de = spacy.load('de')
spacy_en = spacy.load('en')

In [ ]:
sentence = []
label = []
with open('/content/OpenBookQa/train.jsonl') as h:
    for line in h:
        example = json.loads(line)
        print(example.keys())
        # scores = []
        merged_choices = ' A: '.join([choice['text'] for choice in example['question']['choices']])
        input = 'Q: ' + example['question']['stem'] + ' A: ' + merged_choices
        correct_answer = [ choice['text'] for choice in example['question']['choices'] if choice['label'] == example['answerKey'] ][0]
        sentence.append(input)
        label.append(correct_answer)
        #print(input, correct_answer)
dataset_df = pd.DataFrame({'sentence':sentence, 'label':label})

In [ ]:
dataset_df

,sentence,label
0,Q: The sun is responsible for A: puppies learn...,"plants sprouting, blooming and wilting"
1,Q: When standing miles away from Mount Rushmor...,the mountains seem smaller than in photographs
2,Q: When food is reduced in the stomach A: the ...,nutrients are being deconstructed
3,Q: Stars are A: warm lights that float A: made...,great balls of gas burning billions of miles away
4,Q: You can make a telescope with a A: straw A:...,mailing tube
...,...,...
4952,Q: A bulldozer alters the area of A: skyscrape...,skyscrapers
4953,Q: An organism that can survive without the he...,Brewer's yeast
4954,Q: The nimbleness of this animal is a key adap...,the antelope
4955,Q: Birds will have different kinds of beaks de...,organisms they hunt


In [ ]:
#dataset = pd.read_csv("/content/dev.tsv", sep='\t', header=None)
dataset = pd.read_csv("/content/qasc/train.tsv", sep='\t', header=None)
dataset.columns = ["sentence", "label"]

We create the tokenizers.

In [ ]:
def tokenize_de(text):
    """
    Tokenizes German text from a string into a list of strings
    """
    return [tok.text for tok in spacy_de.tokenizer(text)]

def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

The fields remain the same as before.

Load the data.

In [ ]:
from torchtext import data 
import matplotlib.pyplot as plt
Sentence = data.Field(sequential = True, tokenize = 'spacy', batch_first =False, include_lengths=False, lower=False)
Label = data.Field(sequential =True, tokenize ='spacy', is_target=False, batch_first =False, include_lengths=False, lower=False)

In [ ]:
dataset = dataset_df
fields = [('sentence', Sentence),('label',Label)]
example = [data.Example.fromlist([dataset.sentence[i],dataset.label[i]], fields) for i in range(dataset.shape[0])] 
commonqa_ds = data.Dataset(example, fields)
(train, valid) = commonqa_ds.split(split_ratio=[0.85, 0.15], random_state=random.seed(SEED))

In [ ]:
Sentence.build_vocab(train)
Label.build_vocab(train)

In [ ]:
print('Size of input vocab : ', len(Sentence.vocab))
print('Size of label vocab : ', len(Label.vocab))
print('Top 10 words appreared repeatedly :', list(Sentence.vocab.freqs.most_common(10)))
print('Labels : ', Label.vocab.stoi)

Size of input vocab :  10405
Size of label vocab :  4359
Top 10 words appreared repeatedly : [(':', 21131), ('A', 17700), ('a', 4704), ('Q', 4214), ('the', 3677), ('to', 2303), ('of', 2216), ('is', 2037), ('in', 1693), ('?', 1510)]
Labels :  defaultdict(<function _default_unk_index at 0x7efe1c7bfa60>, {'<unk>': 0, '<pad>': 1, 'a': 2, 'the': 3, 'of': 4, 'to': 5, 'in': 6, 'it': 7, 'is': 8, 'and': 9, 'an': 10, 'on': 11, 'be': 12, 'from': 13, 'with': 14, 'water': 15, "'s": 16, 'are': 17, 'will': 18, ',': 19, 'for': 20, 'that': 21, 'their': 22, 'can': 23, 'more': 24, 'its': 25, 'they': 26, 'by': 27, 'light': 28, 'sun': 29, 'up': 30, 'was': 31, '-': 32, 'food': 33, 'air': 34, 'sustenance': 35, 'them': 36, 'energy': 37, 'into': 38, 'star': 39, 'have': 40, 'heat': 41, 'when': 42, 'A': 43, 'down': 44, 'has': 45, 'less': 46, 'earth': 47, 'your': 48, 'away': 49, 'being': 50, 'ice': 51, 'liquid': 52, 'ground': 53, 'as': 54, 'leaves': 55, 'our': 56, 'out': 57, 'plant': 58, 'use': 59, 'H2O': 60, 'at

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
BATCH_SIZE=128
train_iterator, test_iterator = BucketIterator.splits(
    (train, valid), 
    batch_size = BATCH_SIZE, 
    device = device, sort=False, shuffle=False)

Build the vocabulary.

Define the device.

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Create the iterators.

## Building the Seq2Seq Model

### Encoder

First, we'll build the encoder. Similar to the previous model, we only use a single layer GRU, however we now use a *bidirectional RNN*. With a bidirectional RNN, we have two RNNs in each layer. A *forward RNN* going over the embedded sentence from left to right (shown below in green), and a *backward RNN* going over the embedded sentence from right to left (teal). All we need to do in code is set `bidirectional = True` and then pass the embedded sentence to the RNN as before. 

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq8.png?raw=1)

We now have:

$$\begin{align*}
h_t^\rightarrow &= \text{EncoderGRU}^\rightarrow(e(x_t^\rightarrow),h_{t-1}^\rightarrow)\\
h_t^\leftarrow &= \text{EncoderGRU}^\leftarrow(e(x_t^\leftarrow),h_{t-1}^\leftarrow)
\end{align*}$$

Where $x_0^\rightarrow = \text{<sos>}, x_1^\rightarrow = \text{guten}$ and $x_0^\leftarrow = \text{<eos>}, x_1^\leftarrow = \text{morgen}$.

As before, we only pass an input (`embedded`) to the RNN, which tells PyTorch to initialize both the forward and backward initial hidden states ($h_0^\rightarrow$ and $h_0^\leftarrow$, respectively) to a tensor of all zeros. We'll also get two context vectors, one from the forward RNN after it has seen the final word in the sentence, $z^\rightarrow=h_T^\rightarrow$, and one from the backward RNN after it has seen the first word in the sentence, $z^\leftarrow=h_T^\leftarrow$.

The RNN returns `outputs` and `hidden`. 

`outputs` is of size **[src len, batch size, hid dim * num directions]** where the first `hid_dim` elements in the third axis are the hidden states from the top layer forward RNN, and the last `hid_dim` elements are hidden states from the top layer backward RNN. We can think of the third axis as being the forward and backward hidden states concatenated together other, i.e. $h_1 = [h_1^\rightarrow; h_{T}^\leftarrow]$, $h_2 = [h_2^\rightarrow; h_{T-1}^\leftarrow]$ and we can denote all encoder hidden states (forward and backwards concatenated together) as $H=\{ h_1, h_2, ..., h_T\}$.

`hidden` is of size **[n layers * num directions, batch size, hid dim]**, where **[-2, :, :]** gives the top layer forward RNN hidden state after the final time-step (i.e. after it has seen the last word in the sentence) and **[-1, :, :]** gives the top layer backward RNN hidden state after the final time-step (i.e. after it has seen the first word in the sentence).

As the decoder is not bidirectional, it only needs a single context vector, $z$, to use as its initial hidden state, $s_0$, and we currently have two, a forward and a backward one ($z^\rightarrow=h_T^\rightarrow$ and $z^\leftarrow=h_T^\leftarrow$, respectively). We solve this by concatenating the two context vectors together, passing them through a linear layer, $g$, and applying the $\tanh$ activation function. 

$$z=\tanh(g(h_T^\rightarrow, h_T^\leftarrow)) = \tanh(g(z^\rightarrow, z^\leftarrow)) = s_0$$

**Note**: this is actually a deviation from the paper. Instead, they feed only the first backward RNN hidden state through a linear layer to get the context vector/decoder initial hidden state. This doesn't seem to make sense to me, so we have changed it.

As we want our model to look back over the whole of the source sentence we return `outputs`, the stacked forward and backward hidden states for every token in the source sentence. We also return `hidden`, which acts as our initial hidden state in the decoder.

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout):
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional = True)
        
        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, hidden = self.rnn(embedded)
                
        #outputs = [src len, batch size, hid dim * num directions]
        #hidden = [n layers * num directions, batch size, hid dim]
        
        #hidden is stacked [forward_1, backward_1, forward_2, backward_2, ...]
        #outputs are always from the last layer
        
        #hidden [-2, :, : ] is the last of the forwards RNN 
        #hidden [-1, :, : ] is the last of the backwards RNN
        
        #initial decoder hidden is final hidden state of the forwards and backwards 
        #  encoder RNNs fed through a linear layer
        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))
        
        #outputs = [src len, batch size, enc hid dim * 2]
        #hidden = [batch size, dec hid dim]
        
        return outputs, hidden

### Attention

Next up is the attention layer. This will take in the previous hidden state of the decoder, $s_{t-1}$, and all of the stacked forward and backward hidden states from the encoder, $H$. The layer will output an attention vector, $a_t$, that is the length of the source sentence, each element is between 0 and 1 and the entire vector sums to 1.

Intuitively, this layer takes what we have decoded so far, $s_{t-1}$, and all of what we have encoded, $H$, to produce a vector, $a_t$, that represents which words in the source sentence we should pay the most attention to in order to correctly predict the next word to decode, $\hat{y}_{t+1}$. 

First, we calculate the *energy* between the previous decoder hidden state and the encoder hidden states. As our encoder hidden states are a sequence of $T$ tensors, and our previous decoder hidden state is a single tensor, the first thing we do is `repeat` the previous decoder hidden state $T$ times. We then calculate the energy, $E_t$, between them by concatenating them together and passing them through a linear layer (`attn`) and a $\tanh$ activation function. 

$$E_t = \tanh(\text{attn}(s_{t-1}, H))$$ 

This can be thought of as calculating how well each encoder hidden state "matches" the previous decoder hidden state.

We currently have a **[dec hid dim, src len]** tensor for each example in the batch. We want this to be **[src len]** for each example in the batch as the attention should be over the length of the source sentence. This is achieved by multiplying the `energy` by a **[1, dec hid dim]** tensor, $v$.

$$\hat{a}_t = v E_t$$

We can think of $v$ as the weights for a weighted sum of the energy across all encoder hidden states. These weights tell us how much we should attend to each token in the source sequence. The parameters of $v$ are initialized randomly, but learned with the rest of the model via backpropagation. Note how $v$ is not dependent on time, and the same $v$ is used for each time-step of the decoding. We implement $v$ as a linear layer without a bias.

Finally, we ensure the attention vector fits the constraints of having all elements between 0 and 1 and the vector summing to 1 by passing it through a $\text{softmax}$ layer.

$$a_t = \text{softmax}(\hat{a_t})$$

This gives us the attention over the source sentence!

Graphically, this looks something like below. This is for calculating the very first attention vector, where $s_{t-1} = s_0 = z$. The green/teal blocks represent the hidden states from both the forward and backward RNNs, and the attention computation is all done within the pink block.

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq9.png?raw=1)

In [ ]:
class Attention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()
        
        self.attn = nn.Linear((enc_hid_dim * 2) + dec_hid_dim, dec_hid_dim)
        self.v = nn.Linear(dec_hid_dim, 1, bias = False)
        
    def forward(self, hidden, encoder_outputs):
        
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]
        
        batch_size = encoder_outputs.shape[1]
        src_len = encoder_outputs.shape[0]
        
        #repeat decoder hidden state src_len times
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #hidden = [batch size, src len, dec hid dim]
        #encoder_outputs = [batch size, src len, enc hid dim * 2]
        
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim = 2))) 
        
        #energy = [batch size, src len, dec hid dim]

        attention = self.v(energy).squeeze(2)
        
        #attention= [batch size, src len]
        
        return F.softmax(attention, dim=1)

### Decoder

Next up is the decoder. 

The decoder contains the attention layer, `attention`, which takes the previous hidden state, $s_{t-1}$, all of the encoder hidden states, $H$, and returns the attention vector, $a_t$.

We then use this attention vector to create a weighted source vector, $w_t$, denoted by `weighted`, which is a weighted sum of the encoder hidden states, $H$, using $a_t$ as the weights.

$$w_t = a_t H$$

The embedded input word, $d(y_t)$, the weighted source vector, $w_t$, and the previous decoder hidden state, $s_{t-1}$, are then all passed into the decoder RNN, with $d(y_t)$ and $w_t$ being concatenated together.

$$s_t = \text{DecoderGRU}(d(y_t), w_t, s_{t-1})$$

We then pass $d(y_t)$, $w_t$ and $s_t$ through the linear layer, $f$, to make a prediction of the next word in the target sentence, $\hat{y}_{t+1}$. This is done by concatenating them all together.

$$\hat{y}_{t+1} = f(d(y_t), w_t, s_t)$$

The image below shows decoding the first word in an example translation.

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq10.png?raw=1)

The green/teal blocks show the forward/backward encoder RNNs which output $H$, the red block shows the context vector, $z = h_T = \tanh(g(h^\rightarrow_T,h^\leftarrow_T)) = \tanh(g(z^\rightarrow, z^\leftarrow)) = s_0$, the blue block shows the decoder RNN which outputs $s_t$, the purple block shows the linear layer, $f$, which outputs $\hat{y}_{t+1}$ and the orange block shows the calculation of the weighted sum over $H$ by $a_t$ and outputs $w_t$. Not shown is the calculation of $a_t$.

In [ ]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention):
        super().__init__()

        self.output_dim = output_dim
        self.attention = attention
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)
        
        self.fc_out = nn.Linear((enc_hid_dim * 2) + dec_hid_dim + emb_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, encoder_outputs):
             
        #input = [batch size]
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
        
        a = self.attention(hidden, encoder_outputs)
                
        #a = [batch size, src len]
        
        a = a.unsqueeze(1)
        
        #a = [batch size, 1, src len]
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #encoder_outputs = [batch size, src len, enc hid dim * 2]
        
        weighted = torch.bmm(a, encoder_outputs)
        
        #weighted = [batch size, 1, enc hid dim * 2]
        
        weighted = weighted.permute(1, 0, 2)
        
        #weighted = [1, batch size, enc hid dim * 2]
        
        rnn_input = torch.cat((embedded, weighted), dim = 2)
        
        #rnn_input = [1, batch size, (enc hid dim * 2) + emb dim]
            
        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
        
        #output = [seq len, batch size, dec hid dim * n directions]
        #hidden = [n layers * n directions, batch size, dec hid dim]
        
        #seq len, n layers and n directions will always be 1 in this decoder, therefore:
        #output = [1, batch size, dec hid dim]
        #hidden = [1, batch size, dec hid dim]
        #this also means that output == hidden
        assert (output == hidden).all()
        
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)
        
        prediction = self.fc_out(torch.cat((output, weighted, embedded), dim = 1))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden.squeeze(0)

### Seq2Seq

This is the first model where we don't have to have the encoder RNN and decoder RNN have the same hidden dimensions, however the encoder has to be bidirectional. This requirement can be removed by changing all occurences of `enc_dim * 2` to `enc_dim * 2 if encoder_is_bidirectional else enc_dim`. 

This seq2seq encapsulator is similar to the last two. The only difference is that the `encoder` returns both the final hidden state (which is the final hidden state from both the forward and backward encoder RNNs passed through a linear layer) to be used as the initial hidden state for the decoder, as well as every hidden state (which are the forward and backward hidden states stacked on top of each other). We also need to ensure that `hidden` and `encoder_outputs` are passed to the decoder. 

Briefly going over all of the steps:
- the `outputs` tensor is created to hold all predictions, $\hat{Y}$
- the source sequence, $X$, is fed into the encoder to receive $z$ and $H$
- the initial decoder hidden state is set to be the `context` vector, $s_0 = z = h_T$
- we use a batch of `<sos>` tokens as the first `input`, $y_1$
- we then decode within a loop:
  - inserting the input token $y_t$, previous hidden state, $s_{t-1}$, and all encoder outputs, $H$, into the decoder
  - receiving a prediction, $\hat{y}_{t+1}$, and a new hidden state, $s_t$
  - we then decide if we are going to teacher force or not, setting the next input as appropriate

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use teacher forcing 75% of the time
        
        batch_size = src.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #encoder_outputs is all hidden states of the input sequence, back and forwards
        #hidden is the final forward and backward hidden states, passed through a linear layer
        encoder_outputs, hidden = self.encoder(src)
                
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden state and all encoder hidden states
            #receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(input, hidden, encoder_outputs)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1

        return outputs

## Training the Seq2Seq Model

The rest of this session is very similar to the previous one.

We initialise our parameters, encoder, decoder and seq2seq model (placing it on the GPU if we have one). 

In [ ]:
# INPUT_DIM = len(SRC.vocab)
# OUTPUT_DIM = len(TRG.vocab)

INPUT_DIM = len(Sentence.vocab)
OUTPUT_DIM = len(Label.vocab)

ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
ENC_HID_DIM = 512
DEC_HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

model = Seq2Seq(enc, dec, device).to(device)

We use a simplified version of the weight initialization scheme used in the paper. Here, we will initialize all biases to zero and all weights from $\mathcal{N}(0, 0.01)$.

In [ ]:
def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)
            
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(10405, 256)
    (rnn): GRU(256, 512, bidirectional=True)
    (fc): Linear(in_features=1024, out_features=512, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn): Linear(in_features=1536, out_features=512, bias=True)
      (v): Linear(in_features=512, out_features=1, bias=False)
    )
    (embedding): Embedding(4359, 256)
    (rnn): GRU(1280, 512)
    (fc_out): Linear(in_features=1792, out_features=4359, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

Calculate the number of parameters. We get an increase of almost 50% in the amount of parameters from the last model. 

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 18,028,551 trainable parameters


We create an optimizer.

In [ ]:
optimizer = optim.Adam(model.parameters())

We initialize the loss function.

In [ ]:
# TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

TRG_PAD_IDX = Label.vocab.stoi[Label.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

We then create the training loop...

In [ ]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        # src = batch.src
        # trg = batch.trg
        src = batch.sentence
        trg = batch.label
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

...and the evaluation loop, remembering to set the model to `eval` mode and turn off teaching forcing.

In [ ]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            # src = batch.src
            # trg = batch.trg
            src = batch.sentence
            trg = batch.label

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

Finally, define a timing function.

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

Then, we train our model, saving the parameters that give us the best validation loss.

In [ ]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    # valid_loss = evaluate(model, valid_iterator, criterion)
    valid_loss = evaluate(model, test_iterator, criterion)

    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut3-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 6s
	Train Loss: 7.840 | Train PPL: 2539.152
	 Val. Loss: 7.661 |  Val. PPL: 2124.883
Epoch: 02 | Time: 0m 6s
	Train Loss: 7.138 | Train PPL: 1259.407
	 Val. Loss: 7.812 |  Val. PPL: 2468.856
Epoch: 03 | Time: 0m 6s
	Train Loss: 6.993 | Train PPL: 1088.982
	 Val. Loss: 7.921 |  Val. PPL: 2755.148
Epoch: 04 | Time: 0m 6s
	Train Loss: 6.863 | Train PPL: 956.367
	 Val. Loss: 7.950 |  Val. PPL: 2835.825
Epoch: 05 | Time: 0m 6s
	Train Loss: 6.748 | Train PPL: 852.687
	 Val. Loss: 8.391 |  Val. PPL: 4405.527
Epoch: 06 | Time: 0m 6s
	Train Loss: 6.551 | Train PPL: 700.118
	 Val. Loss: 8.256 |  Val. PPL: 3852.382
Epoch: 07 | Time: 0m 6s
	Train Loss: 6.259 | Train PPL: 522.835
	 Val. Loss: 8.483 |  Val. PPL: 4831.521
Epoch: 08 | Time: 0m 6s
	Train Loss: 5.962 | Train PPL: 388.224
	 Val. Loss: 8.788 |  Val. PPL: 6556.995
Epoch: 09 | Time: 0m 6s
	Train Loss: 5.683 | Train PPL: 293.864
	 Val. Loss: 9.038 |  Val. PPL: 8420.232
Epoch: 10 | Time: 0m 6s
	Train Loss: 5.356 | Train P

In [ ]:
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0
)

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.00001, weight_decay=0.01)

In [ ]:
best_valid_loss

6.283

In [ ]:
N_EPOCHS = 30
CLIP = 1
model.load_state_dict(torch.load('tut3-model.pt'))
best_valid_loss = 7.72

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    # valid_loss = evaluate(model, valid_iterator, criterion)
    valid_loss = evaluate(model, test_iterator, criterion)

    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut3-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 6s
	Train Loss: 7.074 | Train PPL: 1180.754
	 Val. Loss: 7.704 |  Val. PPL: 2216.857
Epoch: 02 | Time: 0m 6s
	Train Loss: 7.069 | Train PPL: 1175.278
	 Val. Loss: 7.698 |  Val. PPL: 2203.932
Epoch: 03 | Time: 0m 6s
	Train Loss: 7.067 | Train PPL: 1172.125
	 Val. Loss: 7.692 |  Val. PPL: 2190.730
Epoch: 04 | Time: 0m 6s
	Train Loss: 7.064 | Train PPL: 1168.546
	 Val. Loss: 7.686 |  Val. PPL: 2178.133
Epoch: 05 | Time: 0m 6s
	Train Loss: 7.062 | Train PPL: 1166.215
	 Val. Loss: 7.681 |  Val. PPL: 2166.151
Epoch: 06 | Time: 0m 6s
	Train Loss: 7.059 | Train PPL: 1163.523
	 Val. Loss: 7.675 |  Val. PPL: 2154.457
Epoch: 07 | Time: 0m 6s
	Train Loss: 7.057 | Train PPL: 1161.422
	 Val. Loss: 7.670 |  Val. PPL: 2143.241
Epoch: 08 | Time: 0m 6s
	Train Loss: 7.055 | Train PPL: 1159.035
	 Val. Loss: 7.665 |  Val. PPL: 2132.024
Epoch: 09 | Time: 0m 6s
	Train Loss: 7.054 | Train PPL: 1157.962
	 Val. Loss: 7.660 |  Val. PPL: 2121.814
Epoch: 10 | Time: 0m 6s
	Train Loss: 7.052 | T

In [ ]:
best_valid_loss

7.582783460617065

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.00001, weight_decay=0.01)
N_EPOCHS = 30
CLIP = 1
model.load_state_dict(torch.load('tut3-model.pt'))
best_valid_loss = 7.582783460617065

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    # valid_loss = evaluate(model, valid_iterator, criterion)
    valid_loss = evaluate(model, test_iterator, criterion)

    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut3-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 6s
	Train Loss: 7.034 | Train PPL: 1135.024
	 Val. Loss: 7.580 |  Val. PPL: 1959.310
Epoch: 02 | Time: 0m 6s
	Train Loss: 7.034 | Train PPL: 1134.553
	 Val. Loss: 7.578 |  Val. PPL: 1955.074
Epoch: 03 | Time: 0m 6s
	Train Loss: 7.033 | Train PPL: 1133.601
	 Val. Loss: 7.576 |  Val. PPL: 1950.818
Epoch: 04 | Time: 0m 6s
	Train Loss: 7.033 | Train PPL: 1132.871
	 Val. Loss: 7.574 |  Val. PPL: 1946.699
Epoch: 05 | Time: 0m 6s
	Train Loss: 7.032 | Train PPL: 1132.091
	 Val. Loss: 7.572 |  Val. PPL: 1942.534
Epoch: 06 | Time: 0m 6s
	Train Loss: 7.031 | Train PPL: 1131.387
	 Val. Loss: 7.570 |  Val. PPL: 1938.960
Epoch: 07 | Time: 0m 6s
	Train Loss: 7.031 | Train PPL: 1130.730
	 Val. Loss: 7.568 |  Val. PPL: 1935.690
Epoch: 08 | Time: 0m 6s
	Train Loss: 7.030 | Train PPL: 1130.134
	 Val. Loss: 7.567 |  Val. PPL: 1932.624
Epoch: 09 | Time: 0m 6s
	Train Loss: 7.029 | Train PPL: 1129.162
	 Val. Loss: 7.565 |  Val. PPL: 1929.470
Epoch: 10 | Time: 0m 6s
	Train Loss: 7.029 | T

In [ ]:
best_valid_loss

5.836661656697591

In [ ]:
N_EPOCHS = 50
CLIP = 1
model.load_state_dict(torch.load('tut3-model.pt'))
best_valid_loss = 5.836661656697591

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    # valid_loss = evaluate(model, valid_iterator, criterion)
    valid_loss = evaluate(model, test_iterator, criterion)

    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut3-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 6s
	Train Loss: 7.003 | Train PPL: 1100.311
	 Val. Loss: 7.542 |  Val. PPL: 1886.351
Epoch: 02 | Time: 0m 6s
	Train Loss: 7.001 | Train PPL: 1097.399
	 Val. Loss: 7.543 |  Val. PPL: 1886.982
Epoch: 03 | Time: 0m 6s
	Train Loss: 6.998 | Train PPL: 1094.387
	 Val. Loss: 7.543 |  Val. PPL: 1888.390
Epoch: 04 | Time: 0m 6s
	Train Loss: 6.995 | Train PPL: 1091.196
	 Val. Loss: 7.545 |  Val. PPL: 1890.994
Epoch: 05 | Time: 0m 6s
	Train Loss: 6.992 | Train PPL: 1087.730
	 Val. Loss: 7.547 |  Val. PPL: 1895.485
Epoch: 06 | Time: 0m 6s
	Train Loss: 6.988 | Train PPL: 1084.006
	 Val. Loss: 7.550 |  Val. PPL: 1900.947
Epoch: 07 | Time: 0m 6s
	Train Loss: 6.985 | Train PPL: 1080.017
	 Val. Loss: 7.554 |  Val. PPL: 1907.570
Epoch: 08 | Time: 0m 6s
	Train Loss: 6.981 | Train PPL: 1075.747
	 Val. Loss: 7.558 |  Val. PPL: 1915.272
Epoch: 09 | Time: 0m 6s
	Train Loss: 6.977 | Train PPL: 1071.393
	 Val. Loss: 7.562 |  Val. PPL: 1923.707
Epoch: 10 | Time: 0m 6s
	Train Loss: 6.973 | T

In [ ]:
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 1e-05
    weight_decay: 0.01
)

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=5e-5, weight_decay=0.1)

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=0.1)
N_EPOCHS = 20
CLIP = 1
model.load_state_dict(torch.load('tut3-model.pt'))
best_valid_loss = 6.034162958463033

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    # valid_loss = evaluate(model, valid_iterator, criterion)
    valid_loss = evaluate(model, test_iterator, criterion)

    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut3-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 6s
	Train Loss: 7.051 | Train PPL: 1153.614
	 Val. Loss: 7.532 |  Val. PPL: 1866.087
Epoch: 02 | Time: 0m 6s
	Train Loss: 7.081 | Train PPL: 1188.803
	 Val. Loss: 7.532 |  Val. PPL: 1866.802
Epoch: 03 | Time: 0m 6s
	Train Loss: 7.101 | Train PPL: 1213.427
	 Val. Loss: 7.509 |  Val. PPL: 1824.043
Epoch: 04 | Time: 0m 6s
	Train Loss: 7.121 | Train PPL: 1237.470
	 Val. Loss: 7.480 |  Val. PPL: 1772.238
Epoch: 05 | Time: 0m 6s
	Train Loss: 7.140 | Train PPL: 1261.187
	 Val. Loss: 7.454 |  Val. PPL: 1726.829
Epoch: 06 | Time: 0m 6s
	Train Loss: 7.156 | Train PPL: 1281.329
	 Val. Loss: 7.433 |  Val. PPL: 1691.220
Epoch: 07 | Time: 0m 6s
	Train Loss: 7.168 | Train PPL: 1296.904
	 Val. Loss: 7.418 |  Val. PPL: 1665.573
Epoch: 08 | Time: 0m 6s
	Train Loss: 7.177 | Train PPL: 1308.534
	 Val. Loss: 7.407 |  Val. PPL: 1648.158
Epoch: 09 | Time: 0m 6s
	Train Loss: 7.183 | Train PPL: 1317.174
	 Val. Loss: 7.400 |  Val. PPL: 1636.021
Epoch: 10 | Time: 0m 6s
	Train Loss: 7.188 | T

In [ ]:
best_valid_loss

6.034162958463033

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=0.1)
#optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=0.1)

N_EPOCHS = 20
CLIP = 1
model.load_state_dict(torch.load('tut3-model.pt'))
best_valid_loss = 7.532

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    # valid_loss = evaluate(model, valid_iterator, criterion)
    valid_loss = evaluate(model, test_iterator, criterion)

    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut3-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 6s
	Train Loss: 7.051 | Train PPL: 1153.707
	 Val. Loss: 7.532 |  Val. PPL: 1866.139
Epoch: 02 | Time: 0m 6s
	Train Loss: 7.081 | Train PPL: 1188.758
	 Val. Loss: 7.532 |  Val. PPL: 1866.764
Epoch: 03 | Time: 0m 6s
	Train Loss: 7.101 | Train PPL: 1213.415
	 Val. Loss: 7.509 |  Val. PPL: 1824.059
Epoch: 04 | Time: 0m 6s
	Train Loss: 7.121 | Train PPL: 1237.477
	 Val. Loss: 7.480 |  Val. PPL: 1772.236
Epoch: 05 | Time: 0m 6s
	Train Loss: 7.140 | Train PPL: 1261.177
	 Val. Loss: 7.454 |  Val. PPL: 1726.854
Epoch: 06 | Time: 0m 6s
	Train Loss: 7.156 | Train PPL: 1281.329
	 Val. Loss: 7.433 |  Val. PPL: 1691.231
Epoch: 07 | Time: 0m 6s
	Train Loss: 7.168 | Train PPL: 1296.906
	 Val. Loss: 7.418 |  Val. PPL: 1665.579
Epoch: 08 | Time: 0m 6s
	Train Loss: 7.177 | Train PPL: 1308.536
	 Val. Loss: 7.407 |  Val. PPL: 1648.166
Epoch: 09 | Time: 0m 6s
	Train Loss: 7.183 | Train PPL: 1317.175
	 Val. Loss: 7.400 |  Val. PPL: 1636.027
Epoch: 10 | Time: 0m 6s
	Train Loss: 7.188 | T

In [ ]:
best_valid_loss

7.381375948588054

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=1e-5, weight_decay=0.08)
N_EPOCHS = 50
CLIP = 1
model.load_state_dict(torch.load('tut3-model.pt'))
best_valid_loss = 7.381375948588054

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    # valid_loss = evaluate(model, valid_iterator, criterion)
    valid_loss = evaluate(model, test_iterator, criterion)

    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut3-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 6s
	Train Loss: 7.162 | Train PPL: 1289.133
	 Val. Loss: 7.358 |  Val. PPL: 1569.005
Epoch: 02 | Time: 0m 6s
	Train Loss: 7.159 | Train PPL: 1285.702
	 Val. Loss: 7.362 |  Val. PPL: 1575.030
Epoch: 03 | Time: 0m 6s
	Train Loss: 7.157 | Train PPL: 1283.036
	 Val. Loss: 7.363 |  Val. PPL: 1576.296
Epoch: 04 | Time: 0m 6s
	Train Loss: 7.156 | Train PPL: 1281.178
	 Val. Loss: 7.364 |  Val. PPL: 1578.004
Epoch: 05 | Time: 0m 6s
	Train Loss: 7.154 | Train PPL: 1279.185
	 Val. Loss: 7.365 |  Val. PPL: 1579.579
Epoch: 06 | Time: 0m 6s
	Train Loss: 7.152 | Train PPL: 1277.294
	 Val. Loss: 7.366 |  Val. PPL: 1581.092
Epoch: 07 | Time: 0m 6s
	Train Loss: 7.151 | Train PPL: 1275.513
	 Val. Loss: 7.367 |  Val. PPL: 1582.516
Epoch: 08 | Time: 0m 6s
	Train Loss: 7.150 | Train PPL: 1273.832
	 Val. Loss: 7.368 |  Val. PPL: 1583.842
Epoch: 09 | Time: 0m 6s
	Train Loss: 7.149 | Train PPL: 1272.246
	 Val. Loss: 7.368 |  Val. PPL: 1585.077
Epoch: 10 | Time: 0m 6s
	Train Loss: 7.147 | T

In [ ]:
best_valid_loss

7.358197212219238

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=1e-6, weight_decay=0.1)
N_EPOCHS = 50
CLIP = 1
model.load_state_dict(torch.load('tut3-model.pt'))
best_valid_loss = 7.381375948588054

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    # valid_loss = evaluate(model, valid_iterator, criterion)
    valid_loss = evaluate(model, test_iterator, criterion)

    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut3-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 6s
	Train Loss: 7.161 | Train PPL: 1288.055
	 Val. Loss: 7.358 |  Val. PPL: 1568.311
Epoch: 02 | Time: 0m 6s
	Train Loss: 7.165 | Train PPL: 1293.501
	 Val. Loss: 7.358 |  Val. PPL: 1568.317
Epoch: 03 | Time: 0m 6s
	Train Loss: 7.168 | Train PPL: 1296.988
	 Val. Loss: 7.357 |  Val. PPL: 1567.842
Epoch: 04 | Time: 0m 6s
	Train Loss: 7.169 | Train PPL: 1298.831
	 Val. Loss: 7.357 |  Val. PPL: 1567.491
Epoch: 05 | Time: 0m 6s
	Train Loss: 7.170 | Train PPL: 1299.895
	 Val. Loss: 7.357 |  Val. PPL: 1567.388
Epoch: 06 | Time: 0m 6s
	Train Loss: 7.171 | Train PPL: 1300.526
	 Val. Loss: 7.357 |  Val. PPL: 1567.475
Epoch: 07 | Time: 0m 6s
	Train Loss: 7.171 | Train PPL: 1300.886
	 Val. Loss: 7.357 |  Val. PPL: 1567.691
Epoch: 08 | Time: 0m 6s
	Train Loss: 7.171 | Train PPL: 1301.071
	 Val. Loss: 7.358 |  Val. PPL: 1567.984
Epoch: 09 | Time: 0m 6s
	Train Loss: 7.171 | Train PPL: 1301.145
	 Val. Loss: 7.358 |  Val. PPL: 1568.326
Epoch: 10 | Time: 0m 6s
	Train Loss: 7.171 | T

In [ ]:
best_valid_loss

7.357165654500325

Finally, we test the model on the test set using these "best" parameters.

In [ ]:
model.load_state_dict(torch.load('tut3-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 3.178 | Test PPL:  24.009 |
